<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/ViT_%EA%B8%B0%EB%B0%98_%EB%A9%80%ED%8B%B0%EB%B7%B0_%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import zipfile

# 1. Drive에서 로컬로 복사 (빠름)
shutil.copy("/content/drive/MyDrive/open.zip", "/content/open.zip")

# 2. 압축 풀기
with zipfile.ZipFile("/content/open.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")

# 3. 확인
!ls /content/train


컨티넨탈_10세대_2017_2019	     A_클래스_W177_2020_2025
어코드_10세대_2018_2022		     B_클래스_W246_2013_2018
1시리즈_F20_2013_2015		     뉴_스타일_코란도_C_2017_2019
1시리즈_F20_2016_2019		     프리우스_C_2018_2020
1시리즈_F40_2020_2024		     뉴_CC_2012_2016
그랜드카니발_2006_2010		     CLA_클래스_C117_2014_2019
2008_2015_2017			     CLA_클래스_C118_2020_2025
에쿠스_신형_2010_2015		     CLE_클래스_C236_2024_2025
파나메라_2010_2016		     CLS_클래스_C257_2019_2023
뉴_제타_2011_2016		     CLS_클래스_W218_2012_2017
뉴_카이엔_2011_2018		     아반떼_하이브리드_CN7_2021_2023
엑센트_신형_2011_2019		     아반떼_CN7_2021_2023
스파크_2012_2015		     더_뉴_아반떼_CN7_2023_2025
쿠퍼_컨트리맨_2012_2015		     CT6_2016_2018
올_뉴_모닝_2012_2015		     C_클래스_W204_2008_2015
말리부_2012_2016		     C_클래스_W205_2015_2021
아베오_2012_2016		     C_클래스_W206_2022_2024
뉴_티구안_2012_2016		     제네시스_DH_2014_2016
레이_2012_2017			     쏘나타_DN8_2020_2023
올란도_2012_2018		     쏘나타_디_엣지_DN8_2024_2025
더_뉴_파사트_2012_2019		     e_트론_2020_2023
트랙스_2013_2016		     E_PACE_2018_2020
콰트로포르테_2014_2016		     EQ900_2016_2018
더_뉴_아반떼_2014_2016		     E

In [2]:
import os, glob
import torch
import timm
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# 경로 설정
train_dir = '/content/train'
test_dir = '/content/test'
sample_submission_path = '/content/sample_submission.csv'

# 클래스 정보 로딩
class_names = sorted(os.listdir(train_dir))
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_names)}

# 제출 샘플 불러오기
sample_submission = pd.read_csv(sample_submission_path)


In [3]:
import torch
import gc

def show_memory_status():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # MB 단위
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # MB 단위
    print(f"📊 현재 GPU 메모리 상태: Allocated = {allocated:.2f} MB | Reserved = {reserved:.2f} MB")

# 현재 CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
    print("🔍 초기화 전 GPU 메모리 상태:")
    show_memory_status()

    # GPU 캐시 및 메모리 초기화
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    print("\n🧹 GPU 메모리 초기화 완료")
    print("🔍 초기화 후 GPU 메모리 상태:")
    show_memory_status()
else:
    print("❌ CUDA 사용 불가")


🔍 초기화 전 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB

🧹 GPU 메모리 초기화 완료
🔍 초기화 후 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB


In [4]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset

class MultiViewWithColorDataset(Dataset):
    def __init__(self, image_paths, labels, transform_effnet, transform_vit):
        self.image_paths = image_paths
        self.labels = labels
        self.transform_effnet = transform_effnet
        self.transform_vit = transform_vit

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(path).convert("RGB")
        image_effnet = self.transform_effnet(image)
        image_vit = self.transform_vit(image)

        # ✅ RGB 평균값
        np_img = np.array(image) / 255.0
        mean_rgb = np_img.mean(axis=(0, 1))  # shape: (3,)
        rgb_tensor = torch.tensor(mean_rgb, dtype=torch.float32)

        return {
            'effnet': image_effnet,
            'vit': image_vit,
            'rgb': rgb_tensor
        }, label


In [5]:
from torchvision import transforms

# EfficientNet-B5용 Transform
transform_effnet = transforms.Compose([
    transforms.Resize((456, 456)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

# ViT-B16용 Transform
transform_vit = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],  # ViT 기준
                         [0.5, 0.5, 0.5])
])


In [6]:
import torch.nn as nn
import timm

class HybridEffNetViT(nn.Module):
    def __init__(self, num_classes=396):
        super().__init__()
        self.effnet = timm.create_model("efficientnet_b5", pretrained=True, num_classes=0)
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=0)

        # Feature dims: EffNet = 2048, ViT = 768, RGB 평균값 = 3
        self.classifier = nn.Sequential(
            nn.Linear(2048 + 768 + 3, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )

    def forward(self, inputs):
        eff_feat = self.effnet(inputs['effnet'])  # [B, 2048]
        vit_feat = self.vit(inputs['vit'])        # [B, 768]
        rgb_feat = inputs['rgb']                  # [B, 3]

        combined = torch.cat([eff_feat, vit_feat, rgb_feat], dim=1)
        return self.classifier(combined)


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

class MultiViewImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform_vit=None, transform_eff=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform_vit = transform_vit
        self.transform_eff = transform_eff

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")

        image_vit = self.transform_vit(image)
        image_eff = self.transform_eff(image)
        rgb_mean = self.get_rgb_mean(image)  # RGB 평균값

        return {
            "vit": image_vit,
            "effnet": image_eff,
            "rgb": rgb_mean
        }, label

    @staticmethod
    def get_rgb_mean(image):
        np_img = np.array(image) / 255.0  # [H, W, 3], 정규화
        mean_rgb = np_img.mean(axis=(0, 1))  # [R_mean, G_mean, B_mean]
        return torch.tensor(mean_rgb, dtype=torch.float32)


In [9]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader

# 전체 이미지 경로 및 레이블 리스트 준비
def get_image_paths_and_labels(base_dir):
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(base_dir))
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_names)}
    for cls_name in class_names:
        cls_dir = os.path.join(base_dir, cls_name)
        for fname in os.listdir(cls_dir):
            if fname.endswith(".jpg"):
                image_paths.append(os.path.join(cls_dir, fname))
                labels.append(class_to_idx[cls_name])
    return image_paths, labels, class_to_idx

# 경로에서 데이터 수집
image_paths, labels, class_to_idx = get_image_paths_and_labels(train_dir)

# K-Fold 설정
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Fold 선택 (예: Fold 0만 사용)
for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
    print(f"Using Fold {fold_idx}")

    train_paths = [image_paths[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]
    break  # Fold 0만 사용

# Dataloader 구성 (정확한 변수명 사용)
train_dataset = MultiViewImageDataset(train_paths, train_labels, transform_vit, transform_effnet)
val_dataset = MultiViewImageDataset(val_paths, val_labels, transform_vit, transform_effnet)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)



Using Fold 0


In [10]:
import torch
import gc

def reset_gpu_memory():
    if torch.cuda.is_available():
        print("🔍 초기화 전 GPU 메모리 상태:")
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"  🔸 Allocated: {allocated:.2f} MB | Reserved: {reserved:.2f} MB")

        # 메모리 초기화
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()

        print("🧹 GPU 메모리 초기화 완료")
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"🔍 초기화 후 GPU 메모리 상태: Allocated = {allocated:.2f} MB | Reserved = {reserved:.2f} MB")
    else:
        print("❌ CUDA 사용 불가")


In [11]:
import torch.nn as nn
import timm

class HybridEffNetViT_RGB(nn.Module):
    def __init__(self, num_classes=396):
        super().__init__()

        # EfficientNet-B5 (output: 2048)
        self.effnet = timm.create_model("efficientnet_b5", pretrained=True, num_classes=0)

        # ViT Base (output: 768)
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=0)

        # RGB 평균값: 3개 (R, G, B)
        self.classifier = nn.Sequential(
            nn.Linear(2048 + 768 + 3, 1024),  # 전체 feature vector
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )

    def forward(self, inputs):
        # 입력: 딕셔너리로 구성됨
        eff_feat = self.effnet(inputs['effnet'])  # [B, 2048]
        vit_feat = self.vit(inputs['vit'])        # [B, 768]
        rgb_feat = inputs['rgb']                  # [B, 3]

        combined = torch.cat([eff_feat, vit_feat, rgb_feat], dim=1)  # [B, 2048+768+3]
        return self.classifier(combined)


In [12]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc="Training", leave=False):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(targets).sum().item()
        total += targets.size(0)

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    return epoch_loss, accuracy


def valid_one_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validation", leave=False):
            inputs = {k: v.to(device) for k, v in inputs.items()}
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(targets).sum().item()
            total += targets.size(0)

    epoch_loss = running_loss / len(val_loader)
    accuracy = 100. * correct / total
    return epoch_loss, accuracy


In [13]:
# 예시: smoothing factor = 0.1
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)


In [14]:
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss, model, path):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), path)
            if self.verbose:
                print(f"✅ Validation loss decreased. Saving model to {path}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"⏳ EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


In [15]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        inputs, labels = batch
        for k in inputs:
            inputs[k] = inputs[k].cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(dataloader), correct / total

def valid_one_epoch(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch
            for k in inputs:
                inputs[k] = inputs[k].cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(dataloader), correct / total


In [ ]:
import torch.optim as optim

# 모델, 손실함수, 옵티마이저
model = HybridEffNetViT(num_classes=396).cuda()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

# EarlyStopping 객체
early_stopper = EarlyStopping(patience=5, verbose=True)
save_path = "best_model.pth"

# 학습 시작
EPOCHS = 20
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = valid_one_epoch(model, val_loader, criterion)

    print(f"[Epoch {epoch}] 🟢 Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | 🔵 Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    early_stopper(val_loss, model, save_path)
    if early_stopper.early_stop:
        print("⛔ Early stopping triggered.")
        break


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/122M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

[Epoch 1] 🟢 Train Loss: 5.8623, Acc: 0.0058 | 🔵 Val Loss: 5.5467, Acc: 0.0101
✅ Validation loss decreased. Saving model to best_model.pth


In [ ]:
# Step 9: Test 데이터셋 추론 및 Feature-level TTA + Submission 생성
class TestMultiViewDataset(Dataset):
    def __init__(self, image_paths, transform_effnet, transform_vit):
        self.image_paths = image_paths
        self.transform_effnet = transform_effnet
        self.transform_vit = transform_vit
        self.hflip_effnet = transforms.Compose([
            transforms.Resize((456, 456)),
            transforms.RandomHorizontalFlip(p=1.0),  # Flip
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
        self.hflip_vit = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        image = Image.open(path).convert("RGB")

        # 원본
        effnet_img = self.transform_effnet(image)
        vit_img = self.transform_vit(image)

        # 좌우 반전
        effnet_flip = self.hflip_effnet(image)
        vit_flip = self.hflip_vit(image)

        return {
            "effnet": effnet_img,
            "vit": vit_img,
            "effnet_flip": effnet_flip,
            "vit_flip": vit_flip,
        }


In [ ]:
def inference_tta(model, dataloader):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for batch in dataloader:
            effnet = batch['effnet'].cuda()
            vit = batch['vit'].cuda()
            effnet_flip = batch['effnet_flip'].cuda()
            vit_flip = batch['vit_flip'].cuda()

            # 두 feature 추론 결과 평균
            pred1 = model({"effnet": effnet, "vit": vit})
            pred2 = model({"effnet": effnet_flip, "vit": vit_flip})
            avg_pred = (pred1 + pred2) / 2.0

            preds = avg_pred.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)

    return all_preds


In [ ]:
# 테스트 이미지 경로
test_image_paths = sorted(glob.glob(os.path.join(test_dir, "*.jpg")))

# 테스트 데이터셋
test_dataset = TestMultiViewDataset(test_image_paths, transform_effnet, transform_vit)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

# 모델 로딩 (EarlyStopping에서 저장한 최적 모델)
model.load_state_dict(torch.load("best_model.pth"))
model.cuda()

# 추론
test_preds = inference_tta(model, test_loader)

# 제출 파일 생성
submission = pd.read_csv(sample_submission_path)
submission["label"] = test_preds
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 생성 완료")
